In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import ee
import geemap
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
# 加载研究区域
roi = ee.FeatureCollection('users/311605001111/YangtzeCity/wuhan').geometry()
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,9)

# 加载样本集
samples = ee.FeatureCollection('users/311605001111/SampleDataset/WuhanTrainingPoints_2014')
# Map.addLayer(samples, {}, "WuhanTrainingPoints_2020")
print(samples.size().getInfo())

# # #各类的个数
# clt_water = samples.filter(ee.Filter.eq('waterclass',0))
# print(clt_water.size().getInfo())
# clt_nowater = samples.filter(ee.Filter.eq('waterclass',1))
# print(clt_nowater.size().getInfo())

3986
3277
709


In [4]:
## 加载影像

# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)
# landsat影像可视化参数
visParams = {
    'bands': ['B5', 'B6', 'B4'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}

landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2020-01-01', '2020-12-31') \
    .map(maskL8sr)
Map.addLayer(landsat_images.mosaic().clip(roi),visParams,'landsat image')

In [5]:
# K_mean聚类
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'NDVI','mNDWI', 'AWEI']
clusterer = ee.Clusterer.wekaKMeans(2).train(samples,bands)
result = samples.cluster(clusterer)

print(result.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [113.95369190573179, 30.444606083523855]}, 'id': '000000000000000006ad', 'properties': {'AWEI': 1343.75, 'B2': 787, 'B3': 1004, 'B4': 739, 'B5': 1028, 'B6': 252, 'B7': 133, 'NDVI': 0.16355404258, 'cluster': 1, 'mNDWI': 0.59872609377, 'waterclass': 0}}


In [ ]:
# #聚类后的水体
# clt_water = result.filter(ee.Filter.eq('cluster',0))
# print(clt_water.size().getInfo())
# Map.addLayer(clt_water,{'color': '7CFC00', 'pointSize': 4,'fillColor': '7CFC00'},"clt_water")
# #聚类后的非水体
# clt_nowater = result.filter(ee.Filter.eq('cluster',1))
# print(clt_nowater.size().getInfo())
# Map.addLayer(clt_nowater,{'color': 'f40101ff', 'pointSize': 4,'fillColor': 'ef0b0b56'},"clt_nowater")

In [ ]:
# clt_water = result.filter(ee.Filter.eq('cluster',0)).filter(ee.Filter.eq('waterclass',1))
# print(clt_water.size().getInfo())
# clt_nowater = result.filter(ee.Filter.eq('cluster',1)).filter(ee.Filter.eq('waterclass',0))
# print(clt_nowater.size().getInfo())

In [ ]:
# cluster与waterclass的对比
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft

In [ ]:
# 水体
right = result.map(Compare).filter(ee.Filter.eq('eq',0))
print(right.size().getInfo())
# Map.addLayer(right,{'color': 'FF8C00', 'pointSize': 4,'fillColor': 'FF8C00'},"right")
# 非水体
error = result.map(Compare).filter(ee.Filter.eq('eq',1))
print(error.size().getInfo())
# Map.addLayer(error,{'color': 'FF00FF', 'pointSize': 4,'fillColor': 'FF00FF'},"error")

# 过滤后的样本
filtered_sample = ee.Algorithms.If(right.size().gt(error.size()),right,error)

In [ ]:
## 未经过K_mean聚类
# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','mNDWI','AWEI']
# 分类标签
label = 'waterclass'
# RF训练
trainedClassifier = ee.Classifier.smileRandomForest(20).train(samples,label,bands)

train_accuracy = trainedClassifier.confusionMatrix()
print(train_accuracy.getInfo())
print(train_accuracy.accuracy().getInfo())
print(train_accuracy.kappa().getInfo())

In [ ]:
## 经过K_mean聚类
# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','mNDWI','AWEI']
# 分类标签
label = 'waterclass'
# RF训练
trainedClassifier = ee.Classifier.smileRandomForest(20).train(filtered_sample,label,bands)

train_accuracy = trainedClassifier.confusionMatrix()
print(train_accuracy.getInfo())
print(train_accuracy.accuracy().getInfo())
print(train_accuracy.kappa().getInfo())

In [ ]:
## 样本聚类（K_mean），代码重点内容
# “samples”为样本集
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'NDVI','mNDWI', 'AWEI']
clusterer = ee.Clusterer.wekaKMeans(2).train(samples,bands)
result = samples.cluster(clusterer)

# cluster与waterclass的对比
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft

# 下面二选一
right = result.map(Compare).filter(ee.Filter.eq('eq',0))
error = result.map(Compare).filter(ee.Filter.eq('eq',1))
# 过滤后的样本
filtered_sample = ee.Algorithms.If(right.size().gt(error.size()),right,error)

# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','mNDWI','AWEI']
# 分类标签
label = 'waterclass'
# RF训练
trainedClassifier = ee.Classifier.smileRandomForest(20).train(filtered_sample,label,bands)

In [ ]:
# k_means聚类
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'NDVI','mNDWI', 'AWEI']
# Generic Function to remove a property from a feature
def removeProperty(feat):
    selectProperties = ee.Feature(feat).propertyNames().filter(ee.Filter.neq('item', 'waterclass'))
    return feat.select(selectProperties)
samples = ee.FeatureCollection('users/311605001111/WuhanTrainingPoints_2020')
a = samples.first()
print(a.getInfo())
print(removeProperty(ee.Feature(a)).getInfo())
